In [1]:
import sys
# import mkl
# mkl.set_num_threads(30)

import os
import imp

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

import math

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

import utils
import config
import image
import dataset
import model

from collections import defaultdict
from tqdm import tqdm


imp.reload(config)
imp.reload(image)
imp.reload(utils)
imp.reload(dataset)
imp.reload(model)

<module 'model' from '/home/mlpan/Courses/Capstone-Birds-Action-Recognition/Codes/model.py'>

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Problems

1. When we acquire more than one GPU, tensorflow cannot work.
2. I/O slow
3. When setting exclusive for one node, we may have 2 CPU (16 threading for each). How much we could actually use to avoid OOM? Or how to test it?
4. Warnings?
5. in tensorboard, no learning curve
6. Is that a proper way to use fine-tune?

## Train, Test, Validation Overview

In [3]:
images_metadata_df = dataset.load_images_metadata()
images_metadata_birds_df = images_metadata_df.loc[images_metadata_df['obj_cat_binary']==1].sample(frac = 1, random_state = 42)

images_metadata_dict = dataset.load_images_metadata_dict(images_metadata_birds_df)

dict_dataset = {'train':dict(), 'valid':dict(), 'test':dict()}
for obj_id, obj_info in images_metadata_dict.items():
    ttv_split = obj_info['ttv_split']
    if ttv_split == 1: dict_dataset['train'][obj_id] = obj_info
    elif ttv_split == 2: dict_dataset['valid'][obj_id] = obj_info
    elif ttv_split == 3: dict_dataset['test'][obj_id] = obj_info
    else: raise ValueError()
        
        


In [4]:
len(images_metadata_df['obj_id'].unique())

49255

In [5]:
len(images_metadata_birds_df['obj_id'].unique())

17155

In [6]:
{key:len(value) for key,value in dict_dataset.items()}

{'train': 12005, 'valid': 2576, 'test': 2574}

In [7]:
tf_dataset = {key:tf.data.Dataset.from_generator(lambda :dataset.generator_dataset_activity(value), 
                                       output_types= config.DATASET_TYPE,
                                       output_shapes = config.DATASET_SHAPE) 
              for key,value in dict_dataset.items()}

tf_dataset = {key:dataset.configure_for_performance(value) for key,value in tf_dataset.items()}

del images_metadata_df
del images_metadata_birds_df  

In [ ]:
i3d_model = model.build_model_i3d()

i3d_model.compile(
    optimizer =keras.optimizers.Nadam(lr = 1e-3),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)


i3d_model.fit(
    tf_dataset['train'],
    epochs=10,
    batch_size=config.BATCH_SIZE,
    steps_per_epoch = len(dict_dataset['train']) // config.BATCH_SIZE + 1,
    validation_data=tf_dataset['valid'],
    callbacks = [
                 keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True, monitor = 'val_accuracy'),
                 keras.callbacks.ModelCheckpoint("i3d_ft.h5", save_best_only=True), 
                 keras.callbacks.TensorBoard(utils.get_run_logdir(log_name = 'i3d_ft'))
             ]
)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert

Got `type(handle)`: <class 'tensorflow.python.eager.wrap_function.WrappedFunction'>
Epoch 1/10
161/751 [=====>........................] - ETA: 1:21:41 - loss: 1.7669 - accuracy: 0.5544

In [ ]:
i3d_model_dropout = model.build_model_i3d_dropout()
i3d_model_dropout.compile(
    optimizer =keras.optimizers.Nadam(lr = 1e-3),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

i3d_model_dropout.fit(
    tf_dataset['train'],
    epochs=10,
    batch_size=config.BATCH_SIZE,
    steps_per_epoch = len(dict_dataset['train']) // config.BATCH_SIZE + 1,
    validation_data=tf_dataset['valid'],
    callbacks = [
                 keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True, monitor = 'val_accuracy'),
                 keras.callbacks.ModelCheckpoint("i3d_dropout_ft.h5", save_best_only=True), 
                 keras.callbacks.TensorBoard(utils.get_run_logdir(log_name = 'i3d_dropout_ft'))
             ]
)

In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [9]:
%load_ext tensorboard
%tensorboard --logdir=./logs --port=6012 --host=0.0.0.0

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
